In [1]:
import pandas as pd

from df.keiba_data_frame import KeibaDataFrame
from df.result_frame import ResultFrame
from env.reward import Reward

# Train Data 読み込み
# ※ 既に AI 側で filter/data 整備は実施済みのため再 type 明確化と、特定列を Drop のみ行っている。
kdf = KeibaDataFrame.from_data("./../data/train/2013_train_binary.csv")
kdf.setting_types()

# Reward 準備
reward = Reward.initialize(["./../data/train/2013_result_wide.csv"])

In [2]:
import numpy as np
from tensorflow import keras

new_model = keras.models.load_model("./../model/20230329_binary_model_wide")
new_model.summary()

preds = []

i = 0
for v in list(kdf.groups()):
    v.reset_index(inplace=True, drop=True)
    #obs = v.reindex(range(20), fill_value=-1).values.reshape(1, 1, 20, 51).tolist()
    obs = v[["odds", "pred"]].reindex(range(5), fill_value=-1).values.reshape(1, 1, 5, 2).tolist()
    #print(obs)
    q_values = new_model.predict(obs)
    #print(q_values)
    action = np.argmax(q_values[0])
    #print(action)
    pred_df = pd.DataFrame(
        {
            "track_id": v["track_id"],
            "date_num": v["date_num"],
            "round": v["round"],
            "number": v["number"],
            "pred": v["pred"],
            "action": action,
        }
    )
    preds.append(pred_df)
    # i += 1
    # if i > 100:
    #     break

preds_df = pd.concat(preds)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 128)               1408      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 11)                1419      
                                                                 
Total params: 35,851
Trainable params: 35,851
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 18ms/step


In [3]:
mikounyu_count = 0
kounyu_count = 0
atari_count = 0
atari_reward = 0
hazure_count = 0
reward_total = 0

action_0_count = 0
action_1_count = 0
action_2_count = 0
action_3_count = 0
action_4_count = 0
action_5_count = 0
action_6_count = 0
action_7_count = 0
action_8_count = 0
action_9_count = 0

df = preds_df.sort_values(
    ["date_num", "track_id", "round"], ascending=[True, True, True]
)

for _, v in df.groupby(["date_num", "track_id", "round"], sort=False):
    action = int(v.iloc[0]["action"])

    if action == 0:
        action_0_count += 1
    elif action == 1:
        action_1_count += 1
    elif action == 2:
        action_2_count += 1
    elif action == 3:
        action_3_count += 1
    elif action == 4:
        action_4_count += 1
    elif action == 5:
        action_5_count += 1
    elif action == 6:
        action_6_count += 1
    elif action == 7:
        action_7_count += 1
    elif action == 8:
        action_8_count += 1
    elif action == 9:
        action_9_count += 1

    if action == 10:
        reward_total += 0
        mikounyu_count += 1
    else:
        kounyu_count += 1
        result = reward.calc(v, action)
        if result > 0:
            atari_count += 1
            atari_reward += result + 1000
        else:
            hazure_count += 1

        reward_total += result

print(f"RANK_ONE_TWO_HORSE 回数: {action_0_count}")
print(f"RANK_ONE_THREE_HORSE 回数: {action_1_count}")
print(f"RANK_ONE_FOUR_HORSE 回数: {action_2_count}")
print(f"RANK_ONE_FIVE_HORSE 回数: {action_3_count}")
print(f"RANK_TWO_THREE_HORSE 回数: {action_4_count}")
print(f"RANK_TWO_FOUR_HORSE 回数: {action_5_count}")
print(f"RANK_TWO_FIVE_HORSE 回数: {action_6_count}")
print(f"RANK_THREE_FOUR_HORSE 回数: {action_7_count}")
print(f"RANK_THREE_FIVE_HORSE 回数: {action_8_count}")
print(f"RANK_FOUR_FIVE_HORSE 回数: {action_9_count}")
print(f"未購入回数: {mikounyu_count}")
print()
print(f"当たり馬券: {atari_count}")
print(f"はずれ馬券: {hazure_count}")
print(f"的中率: {(atari_count / (atari_count + hazure_count))}")
print()
print(f"当たり金額: {atari_reward}")
print(f"購入金額: {((atari_count + hazure_count) * 1000)}")
print(f"儲け: {reward_total}")
print(f"回収率: {(atari_reward / ((atari_count + hazure_count) * 1000))}")

RANK_ONE_TWO_HORSE 回数: 210
RANK_ONE_THREE_HORSE 回数: 175
RANK_ONE_FOUR_HORSE 回数: 219
RANK_ONE_FIVE_HORSE 回数: 166
RANK_TWO_THREE_HORSE 回数: 135
RANK_TWO_FOUR_HORSE 回数: 93
RANK_TWO_FIVE_HORSE 回数: 73
RANK_THREE_FOUR_HORSE 回数: 128
RANK_THREE_FIVE_HORSE 回数: 57
RANK_FOUR_FIVE_HORSE 回数: 56
未購入回数: 2142

当たり馬券: 291
はずれ馬券: 1021
的中率: 0.22179878048780488

当たり金額: 1084700
購入金額: 1312000
儲け: -227300
回収率: 0.8267530487804878
